In [1]:
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library(pheatmap))
suppressPackageStartupMessages(library(stringr))
library(hise)
library(plyr)
library(purrr)
library(vegan)
#library(arrow)
library(rstatix)
library(parallel)
library(ggpubr)
library(ComplexHeatmap)
library(compositions)
library(ggsci)
library(ggpubr)

------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: ‘plyr’


The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



Attaching package: ‘purrr’


The following object is masked from ‘package:plyr’:

    compact


Loading required package: permute

Loading required package: lattice

This is vegan 2.6-4


Attaching package: ‘rstatix’


The following objects are masked from ‘package:plyr’:

    desc, mutate


The following object is masked from ‘package:stats’:

    filter


Loading required package: ggplot2


Attaching package: ‘ggpubr’


The following object is masked from ‘package:plyr’:

    muta

ERROR: Error in library(compositions): there is no package called ‘compositions’


In [3]:
colors = c('#FDC086','#FFFF99','#F0027F','#BF5B17','#666666',
          '#1B9E77','#D95F02','#7570B3','#E7298A','#66A61E','#E6AB02','#A6761D','#666666',
          '#A6CEE3','#1F78B4','#B2DF8A','#33A02C','#FB9A99','#E31A1C','#FDBF6F','#FF7F00',
          '#CAB2D6','#6A3D9A','#FFFF99','#B15928','#FBB4AE','#B3CDE3','#CCEBC5','#DECBE4',
          '#FED9A6','#FFFFCC','#E5D8BD','#FDDAEC','#F2F2F2','#B3E2CD','#FDCDAC','#CBD5E8',
          '#F4CAE4','#E6F5C9','#FFF2AE','#F1E2CC','#CCCCCC','#E41A1C','#377EB8','#4DAF4A',
          '#984EA3','#FF7F00','#FFFF33','#A65628','#F781BF','#999999','#66C2A5','#FC8D62',
          '#8DA0CB','#E78AC3','#A6D854','#FFD92F','#E5C494','#B3B3B3','#8DD3C7','#FFFFB3',
          '#BEBADA','#FB8072','#80B1D3','#FDB462','#B3DE69','#FCCDE5','#D9D9D9','#BC80BD',
          '#CCEBC5','#FFED6F')


# Reading the file

In [4]:
# Read the CSV file into a data frame
all_df <- read.csv('/home/jupyter/Myeloid_cells/files/Parquet_files/AIFI_Reference_processed.csv')


In [5]:
#remove UPI from the cohort
all_df <- all_df %>% filter(cohort.cohortGuid != 'UP1')

In [6]:
all_df <- all_df %>%
  filter(!(AIFI_L1 %in% c('ILC', 'Erythrocyte', 'Platelet')))

In [7]:
#colnames(myeloid_df)

In [8]:
rownames(all_df)<-all_df$cell_uuid

In [9]:
unique(all_df$cohort.cohortGuid)

[1] "BR1" "BR2"

# CLR Transform


In [10]:
clr_transform <- function(x) {
  if (length(x) == 0) {
    return(NA)  # return NA for empty vectors
  }
  geom_mean <- exp(mean(log(x)))
  return(log(x / geom_mean))
}

# Comparsion within total PBMCs (Percentage and Percentage CLR)

In [11]:
## Percentage of total

result <- all_df %>%
  dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
  dplyr::summarise(count = n()) %>%                  # Count occurrences
  dplyr::group_by(subject.subjectGuid) %>%                       
  dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
  dplyr::group_by(subject.subjectGuid) %>%     
  dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
  dplyr::ungroup()

head(result)
#sum(result$subject.subjectGuid)

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


AIFI_L3,subject.subjectGuid,count,percentage,percentage_clr
<chr>,<chr>,<int>,<dbl>,<dbl>
ASDC,BR1001,6,0.03248863,-2.284753
ASDC,BR1002,5,0.02645363,-2.298168
ASDC,BR1003,4,0.02099958,-2.834614
ASDC,BR1004,3,0.01721763,-2.657373
ASDC,BR1005,5,0.02821352,-1.969456
ASDC,BR1006,4,0.02308536,-2.394691


In [12]:
result %>% filter(subject.subjectGuid == "BR1001") %>% 
      select(percentage) %>%
      pull() %>% 
      sum()

[1] 100

In [13]:
result<-left_join(result,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))

In [ ]:
head(result)


In [ ]:
table(result$AIFI_L3)

# Specific cell types (Parent)

In [ ]:
cell = "T cell"

In [ ]:
cell_df <- all_df %>% dplyr::filter(AIFI_L1 == cell)


In [ ]:
head(cell_df)

In [ ]:
result_final <- cell_df %>%
  dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
  dplyr::summarise(count = n()) %>%                  # Count occurrences
  dplyr::group_by(subject.subjectGuid) %>%                       
  dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
  dplyr::group_by(subject.subjectGuid) %>%     
  dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
  dplyr::ungroup()




In [ ]:
head(result_final)

In [ ]:
result_final<-left_join(result_final,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))

In [ ]:
result_final$facet_combined <- paste(result_final$cohort.cohortGuid, result_final$CMV.IgG.Serology.Result.Interpretation, sep = "_")
result_final$facet_combined<- factor(result_final$facet_combined)

In [ ]:
head(result_final)

In [ ]:
#New loop

In [ ]:
cell_L3 = "CD4 MAIT"

In [ ]:
L3_df <- result_final %>% dplyr::filter(AIFI_L3 == "CD4 MAIT")

In [ ]:
head(L3_df)

In [ ]:
stat_test <- wilcox_test(percentage ~ facet_combined, data = L3_df, comparisons = my_comparisons, paired= FALSE)%>% 
       adjust_pvalue(method = 'BH') %>% 
       add_xy_position(x = "facet_combined", dodge = 0.8)

In [ ]:
head(stat_test)

In [ ]:
 outfile <- paste("/home/jupyter/Myeloid_cells/Plots/CLR_Comparisons_cohort_&_CMV_Parent/CLR_Comparisons_cohort_&_CMV_parent_TEST", cell_L3, ".pdf", sep="")
    options(repr.plot.width = 18, repr.plot.height = 10)
    p3 <- ggplot(L3_df, aes(x = facet_combined, y = percentage_clr, color = CMV.IgG.Serology.Result.Interpretation)) +
      geom_boxplot(width = 0.5, outlier.shape = NA) +
      geom_jitter(width =0.1) +
      ylab('percentage_clr') + 
      xlab(cell) +
      scale_color_manual(values = c("green", "red", "orange", "blue")) +
      theme(axis.text.x = element_text(size = 12, angle = 90 ),     # Adjust font size of x-axis text
            axis.text.y = element_text(size = 12),     # Adjust font size of y-axis text
            axis.title = element_text(size = 14),
            panel.background = element_rect(fill = "white"),
            axis.line = element_line(color = "black"),  # Set axis line color to black,  # Set major grid line color to black
            panel.grid.minor = element_blank(),
            legend.position = "top",  # Set legend position to top
            #legend.title = element_blank()
           ) +
      geom_boxplot(position = position_dodge(width = 1))+
      stat_pvalue_manual(stat_test, label = "p.adj.signif", tip.length = 0) +
      scale_y_continuous(expand = expansion(mult = c(0, 0.1)))
      pdf(file = outfile, width = 7, height = 11)
      print(p3)
      dev.off()



# Final Loop code

In [14]:
unique_AIFI_L1 <- unique(all_df$AIFI_L1)
unique_AIFI_L1
#unique_AIFI_L3 <- unique(result_final$AIFI_L3)
#unique_AIFI_L3

[1] "T cell"          "NK cell"         "Monocyte"        "DC"             
[5] "Progenitor cell" "B cell"

## CMV and Age grouping

In [15]:
##For CLR - PARENT PLOT

my_comparisons <- list(c("BR1_Negative", "BR2_Negative"), c("BR1_Negative", "BR1_Positive"), c("BR1_Positive", "BR2_Positive"), c("BR2_Negative", "BR2_Positive") )

for(cell in unique_AIFI_L1){
    
    print(cell)
    
    #rm(cell_df, result_final)
    
    cell_df <- all_df %>% dplyr::filter(AIFI_L1 == cell)

    
    result_final <- cell_df %>%
      dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
      dplyr::summarise(count = n()) %>%                  # Count occurrences
      dplyr::group_by(subject.subjectGuid) %>%                       
      dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
      dplyr::group_by(subject.subjectGuid) %>%     
      dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
      dplyr::ungroup()

    
    result_final<-left_join(result_final,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))
    
    result_final$facet_combined <- paste(result_final$cohort.cohortGuid, result_final$CMV.IgG.Serology.Result.Interpretation, sep = "_")
    result_final$facet_combined<- factor(result_final$facet_combined)
    
    for(cell_L3 in unique(result_final$AIFI_L3)){
        
        #rm(L3_df)
        
        L3_df <- result_final %>% dplyr::filter(AIFI_L3 == cell_L3)
        
        stat_test <- wilcox_test(percentage_clr ~ facet_combined, data = L3_df, comparisons = my_comparisons, paired= FALSE)%>% 
               adjust_pvalue(method = 'BH') %>% 
               add_xy_position(x = "facet_combined", dodge = 0.8)




        outfile <- paste("/home/jupyter/Myeloid_cells/Plots/CLR_Comparisons_cohort_&_CMV_Parent/CLR_Comparisons_cohort_&_CMV_parent_", cell_L3, ".pdf", sep="")
        options(repr.plot.width = 18, repr.plot.height = 10)
        p3 <- ggplot(L3_df, aes(x = facet_combined, y = percentage_clr, color = CMV.IgG.Serology.Result.Interpretation)) +
          geom_boxplot(width = 0.5, outlier.shape = NA) +
          geom_jitter(width =0.1) +
          ylab('percentage_clr') + 
          xlab(cell_L3) +
          scale_color_manual(values = c("green", "red", "orange", "blue")) +
          theme(axis.text.x = element_text(size = 12, angle = 90 ),     # Adjust font size of x-axis text
                axis.text.y = element_text(size = 12),     # Adjust font size of y-axis text
                axis.title = element_text(size = 14),
                panel.background = element_rect(fill = "white"),
                axis.line = element_line(color = "black"),  # Set axis line color to black,  # Set major grid line color to black
                panel.grid.minor = element_blank(),
                legend.position = "top",  # Set legend position to top
                #legend.title = element_blank()
               ) +
          geom_boxplot(position = position_dodge(width = 1))+
          stat_pvalue_manual(stat_test, label = "p.adj.signif", tip.length = 0) +
          scale_y_continuous(expand = expansion(mult = c(0, 0.1)))
          pdf(file = outfile, width = 7, height = 11)
          print(p3)
          dev.off()

        
        }
    
    
    }





[1] "T cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "NK cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Monocyte"


...

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Progenitor cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "B cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


In [16]:
##For Percentage - PARENT PLOT

my_comparisons <- list(c("BR1_Negative", "BR2_Negative"), c("BR1_Negative", "BR1_Positive"), c("BR1_Positive", "BR2_Positive"), c("BR2_Negative", "BR2_Positive") )

for(cell in unique_AIFI_L1){
    
    print(cell)
    
    #rm(cell_df, result_final)
    
    cell_df <- all_df %>% dplyr::filter(AIFI_L1 == cell)

    
    result_final <- cell_df %>%
      dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
      dplyr::summarise(count = n()) %>%                  # Count occurrences
      dplyr::group_by(subject.subjectGuid) %>%                       
      dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
      dplyr::group_by(subject.subjectGuid) %>%     
      dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
      dplyr::ungroup()

    
    result_final<-left_join(result_final,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))
    
    result_final$facet_combined <- paste(result_final$cohort.cohortGuid, result_final$CMV.IgG.Serology.Result.Interpretation, sep = "_")
    result_final$facet_combined<- factor(result_final$facet_combined)
    
    for(cell_L3 in unique(result_final$AIFI_L3)){
        
        #rm(L3_df)
        
        L3_df <- result_final %>% dplyr::filter(AIFI_L3 == cell_L3)
        
        stat_test <- wilcox_test(percentage ~ facet_combined, data = L3_df, comparisons = my_comparisons, paired= FALSE)%>% 
               adjust_pvalue(method = 'BH') %>% 
               add_xy_position(x = "facet_combined", dodge = 0.8)




        outfile <- paste("/home/jupyter/Myeloid_cells/Plots/Percentage_Comparisons_cohort_&_CMV_Parent/Percentage_Comparisons_cohort_&_CMV_parent_", cell_L3, ".pdf", sep="")
        options(repr.plot.width = 18, repr.plot.height = 10)
        p3 <- ggplot(L3_df, aes(x = facet_combined, y = percentage, color = CMV.IgG.Serology.Result.Interpretation)) +
          geom_boxplot(width = 0.5, outlier.shape = NA) +
          geom_jitter(width =0.1) +
          ylab('percentage') + 
          xlab(cell_L3) +
          scale_color_manual(values = c("green", "red", "orange", "blue")) +
          theme(axis.text.x = element_text(size = 12, angle = 90 ),     # Adjust font size of x-axis text
                axis.text.y = element_text(size = 12),     # Adjust font size of y-axis text
                axis.title = element_text(size = 14),
                panel.background = element_rect(fill = "white"),
                axis.line = element_line(color = "black"),  # Set axis line color to black,  # Set major grid line color to black
                panel.grid.minor = element_blank(),
                legend.position = "top",  # Set legend position to top
                #legend.title = element_blank()
               ) +
          geom_boxplot(position = position_dodge(width = 1))+
          stat_pvalue_manual(stat_test, label = "p.adj.signif", tip.length = 0) +
          scale_y_continuous(expand = expansion(mult = c(0, 0.1)))
          pdf(file = outfile, width = 7, height = 11)
          print(p3)
          dev.off()

        
        }
    
    
    }





[1] "T cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "NK cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Monocyte"


...

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Progenitor cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "B cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


## SEX and AGE grouping

In [15]:
##For CLR - PARENT PLOT

my_comparisons <- list(c("BR1_Female", "BR2_Female"), c("BR1_Female", "BR1_Male"), c("BR1_Male", "BR2_Male"), c("BR2_Female", "BR2_Male") )

for(cell in unique_AIFI_L1){
    
    print(cell)
    
    #rm(cell_df, result_final)
    
    cell_df <- all_df %>% dplyr::filter(AIFI_L1 == cell)

    
    result_final <- cell_df %>%
      dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
      dplyr::summarise(count = n()) %>%                  # Count occurrences
      dplyr::group_by(subject.subjectGuid) %>%                       
      dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
      dplyr::group_by(subject.subjectGuid) %>%     
      dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
      dplyr::ungroup()

    
    result_final<-left_join(result_final,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))
    
    result_final$facet_combined <- paste(result_final$cohort.cohortGuid, result_final$subject.biologicalSex, sep = "_")
    result_final$facet_combined<- factor(result_final$facet_combined)
    
    for(cell_L3 in unique(result_final$AIFI_L3)){
        
        #rm(L3_df)
        
        L3_df <- result_final %>% dplyr::filter(AIFI_L3 == cell_L3)
        
        stat_test <- wilcox_test(percentage_clr ~ facet_combined, data = L3_df, comparisons = my_comparisons, paired= FALSE)%>% 
               adjust_pvalue(method = 'BH') %>% 
               add_xy_position(x = "facet_combined", dodge = 0.8)




        outfile <- paste("/home/jupyter/Myeloid_cells/Plots/CLR_Comparisons_cohort_&_Sex_Parent/CLR_Comparisons_cohort_&_Sex_parent_", cell_L3, ".pdf", sep="")
        options(repr.plot.width = 18, repr.plot.height = 10)
        p3 <- ggplot(L3_df, aes(x = facet_combined, y = percentage_clr, color = subject.biologicalSex)) +
          geom_boxplot(width = 0.5, outlier.shape = NA) +
          geom_jitter(width =0.1) +
          ylab('percentage_clr') + 
          xlab(cell_L3) +
          scale_color_manual(values = c("red", "blue")) +
          theme(axis.text.x = element_text(size = 12, angle = 90 ),     # Adjust font size of x-axis text
                axis.text.y = element_text(size = 12),     # Adjust font size of y-axis text
                axis.title = element_text(size = 14),
                panel.background = element_rect(fill = "white"),
                axis.line = element_line(color = "black"),  # Set axis line color to black,  # Set major grid line color to black
                panel.grid.minor = element_blank(),
                legend.position = "top",  # Set legend position to top
                #legend.title = element_blank()
               ) +
          geom_boxplot(position = position_dodge(width = 1))+
          stat_pvalue_manual(stat_test, label = "p.adj.signif", tip.length = 0) +
          scale_y_continuous(expand = expansion(mult = c(0, 0.1)))
          pdf(file = outfile, width = 7, height = 11)
          print(p3)
          dev.off()

        
        }
    
    
    }





[1] "T cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "NK cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Monocyte"


...

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Progenitor cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "B cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


In [16]:
##For Percentage - PARENT PLOT

my_comparisons <- list(c("BR1_Female", "BR2_Female"), c("BR1_Female", "BR1_Male"), c("BR1_Male", "BR2_Male"), c("BR2_Female", "BR2_Male") )

for(cell in unique_AIFI_L1){
    
    print(cell)
    
    #rm(cell_df, result_final)
    
    cell_df <- all_df %>% dplyr::filter(AIFI_L1 == cell)

    
    result_final <- cell_df %>%
      dplyr::group_by(AIFI_L3, subject.subjectGuid) %>%               
      dplyr::summarise(count = n()) %>%                  # Count occurrences
      dplyr::group_by(subject.subjectGuid) %>%                       
      dplyr::mutate(percentage = (count / sum(count)) * 100) %>% 
      dplyr::group_by(subject.subjectGuid) %>%     
      dplyr::mutate(percentage_clr = clr_transform((percentage))) %>% # Group again by subjectGuid for CLR# Calculate percentage
      dplyr::ungroup()

    
    result_final<-left_join(result_final,unique(all_df[c('cohort.cohortGuid','subject.subjectGuid','subject.biologicalSex','CMV.IgG.Serology.Result.Interpretation')]),by=c('subject.subjectGuid'))
    
    result_final$facet_combined <- paste(result_final$cohort.cohortGuid, result_final$subject.biologicalSex, sep = "_")
    result_final$facet_combined<- factor(result_final$facet_combined)
    
    for(cell_L3 in unique(result_final$AIFI_L3)){
        
        #rm(L3_df)
        
        L3_df <- result_final %>% dplyr::filter(AIFI_L3 == cell_L3)
        
        stat_test <- wilcox_test(percentage ~ facet_combined, data = L3_df, comparisons = my_comparisons, paired= FALSE)%>% 
               adjust_pvalue(method = 'BH') %>% 
               add_xy_position(x = "facet_combined", dodge = 0.8)




        outfile <- paste("/home/jupyter/Myeloid_cells/Plots/Percentage_Comparisons_cohort_&_Sex_Parent/Percentage_Comparisons_cohort_&_Sex_parent_", cell_L3, ".pdf", sep="")
        options(repr.plot.width = 18, repr.plot.height = 10)
        p3 <- ggplot(L3_df, aes(x = facet_combined, y = percentage, color = subject.biologicalSex)) +
          geom_boxplot(width = 0.5, outlier.shape = NA) +
          geom_jitter(width =0.1) +
          ylab('percentage') + 
          xlab(cell_L3) +
          scale_color_manual(values = c("red", "blue")) +
          theme(axis.text.x = element_text(size = 12, angle = 90 ),     # Adjust font size of x-axis text
                axis.text.y = element_text(size = 12),     # Adjust font size of y-axis text
                axis.title = element_text(size = 14),
                panel.background = element_rect(fill = "white"),
                axis.line = element_line(color = "black"),  # Set axis line color to black,  # Set major grid line color to black
                panel.grid.minor = element_blank(),
                legend.position = "top",  # Set legend position to top
                #legend.title = element_blank()
               ) +
          geom_boxplot(position = position_dodge(width = 1))+
          stat_pvalue_manual(stat_test, label = "p.adj.signif", tip.length = 0) +
          scale_y_continuous(expand = expansion(mult = c(0, 0.1)))
          pdf(file = outfile, width = 7, height = 11)
          print(p3)
          dev.off()

        
        }
    
    
    }





[1] "T cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "NK cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Monocyte"


...

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "Progenitor cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.


[1] "B cell"


`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.
